In [1]:
import numpy as np

In [2]:
data_1 = np.genfromtxt("hmm_pb1.csv",delimiter=',')
data_1 = data_1.astype(int)

In [3]:
len(data_1)

275

In [4]:
def b(num,dice_type='L'):
    if dice_type == 'L':
        prob = [1/10,1/10,1/10,1/10,1/10,0.5]
        return prob[num-1]
    else:
        prob = [1/6,1/6,1/6,1/6,1/6,1/6]
        return prob[num-1]

In [5]:
def Viterbi(data,b):
    #Set up matrix C
    C = np.zeros(shape=(len(data),2))
    #Placeholder for pointers
    ptr = np.zeros(shape=(len(data),2))
    
    T = len(data)
    
    #Initialization
    pi_k = 0.5
    C[0][0] = np.log(pi_k*b(data[0],'F'))
    C[0][1] = np.log(pi_k*b(data[0],'L'))
    ptr[0][0],ptr[0][1]  = 0,1
    
    # For t=2 to T
    for t in range(1,len(data)):
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM FAIR DICE
        First_term  = np.log(b(data[t],'F'))
        Second_term = max(C[t-1][0]+np.log(0.95),C[t-1][1]+np.log(0.05))
        C[t][0] = First_term + Second_term
        
        #For pointer
        temp_list = np.array([C[t-1][0]+np.log(0.95),C[t-1][1]+np.log(0.05)])
        ptr[t][0]  = np.argmax(temp_list)
        
        
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM LOADED DICE
        First_term  = np.log(b(data[t],'L'))
        Second_term = max(C[t-1][0]+np.log(0.05),C[t-1][1]+np.log(0.95))
        C[t][1] = First_term + Second_term
        
        #For pointer
        temp_list = np.array([C[t-1][0]+np.log(0.05),C[t-1][1]+np.log(0.95)])
        ptr[t][1]  = np.argmax(temp_list)
    
   
    #Get the sequence
    y_T = np.argmax(C[T-1])
    sequence = np.zeros(shape=(len(data),1))
    sequence[-1] = ptr[-1][y_T]
    for i in range(len(data)-1,0,-1):
        
        sequence[i-1] = ptr[i][int(sequence[i])]
        
    return (sequence+1).reshape(1,-1)

In [6]:
sequence = Viterbi(data_1,b)

In [7]:
sequence

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 

In [8]:
def forward(data,b):
    #Set up matrix alpha
    alpha = np.zeros(shape=(len(data),2))
    
    T = len(data)
    
    #Initialization
    pi_k = 0.5
    alpha[0][0] = pi_k*b(data[0],'F')
    alpha[0][1] = pi_k*b(data[0],'L')
    
    # For t=2 to T
    for t in range(1,len(data)):
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM FAIR DICE
        b_term  = b(data[t],'F')
        summation_term  = alpha[t-1][0]*0.95+alpha[t-1][1]*0.05
        alpha[t][0] = b_term*summation_term
        
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM LOADED DICE
        b_term  = b(data[t],'L')
        summation_term  = alpha[t-1][0]*0.05+alpha[t-1][1]*0.95
        alpha[t][1] = b_term*summation_term
    
   
    print(alpha[124][0]/alpha[124][1])
    return alpha

In [9]:
_ = forward(data_1,b)

0.39891275119173036


In [10]:
def backward(data,b):
    #Set up matrix alpha
    beta = np.zeros(shape=(len(data),2))
    
    T = len(data)-1 #Since python is 0-based
    
    #Initialization
    beta[-1][0] = 1
    beta[-1][1] = 1
    
    # For t=2 to T
    for t in range(T-1,0,-1):
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM FAIR DICE
        beta[t][0] = 0.95*b(data[t+1],'F')*beta[t+1][0] + 0.05*b(data[t+1],'L')*beta[t+1][1] 
        
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM LOADED DICE
        beta[t][1] = 0.95*b(data[t+1],'L')*beta[t+1][1] + 0.05*b(data[t+1],'F')*beta[t+1][0]
    
   
    print(beta[124][0]/beta[124][1])
    return beta
    

In [11]:
_ = backward(data_1,b)

3.8564482012611956


# part 2

In [12]:
data_2 = np.genfromtxt("hmm_pb2.csv",delimiter=',')
data_2 = data_2.astype(int)

In [13]:
def forwardWB(data,ProbEmission,ProbTransition,alpha,pi):
    
    T = len(data)
    
    #Normalize Initialization
    alpha[0][0] = pi[0]*ProbEmission[0][data[0]-1]
    alpha[0][1] = pi[1]*ProbEmission[1][data[0]-1]
    alpha[0][0],alpha[0][1] = alpha[0][0]/(alpha[0][0]+alpha[0][1]),alpha[0][1]/(alpha[0][0]+alpha[0][1])
    
    # For t=2 to T
    for t in range(1,len(data)):
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM FAIR DICE
        b_term  = ProbEmission[0][data[t]-1]
        summation_term  = alpha[t-1][0]*ProbTransition[0][0]+alpha[t-1][1]*ProbTransition[0][1]
        alpha[t][0] = b_term*summation_term
        
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM LOADED DICE
        b_term  = ProbEmission[1][data[t]-1]
        summation_term  = alpha[t-1][0]*ProbTransition[1][0]+alpha[t-1][1]*ProbTransition[1][1]
        alpha[t][1] = b_term*summation_term
        
        # Normalization step
        alpha[t][0],alpha[t][1] = alpha[t][0]/(alpha[t][0]+alpha[t][1]),alpha[t][1]/(alpha[t][0]+alpha[t][1])
        
    #print(alpha[124][0]/alpha[124][1])
    #print(np.all(np.sum(alpha,axis=1))==1)
    #print(alpha[:5])
    return alpha

In [14]:
def backwardWB(data,ProbEmission,ProbTransition,beta,pi):
    
    T = len(data)-1 #Since python is 0-based
    
    #Normalize Initialization
    beta[-1][0] = 0.5
    beta[-1][1] = 0.5
    
    # For t=2 to T
    for t in range(T-1,-1,-1):
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM FAIR DICE
        #beta[t][0] = 0.95*b(data[t+1],'F')*beta[t+1][0] + 0.05*b(data[t+1],'L')*beta[t+1][1] 
        
        beta[t][0] =  ProbTransition[0][0]*ProbEmission[0][data[t+1]-1]*beta[t+1][0] \
                      + ProbTransition[0][1]*ProbEmission[1][data[t+1]-1]*beta[t+1][1] 
        
        #CASE WHERE WE ARE CONSIDERING THAT WE DRAW FROM LOADED DICE
        #beta[t][1] = 0.95*b(data[t+1],'L')*beta[t+1][1] + 0.05*b(data[t+1],'F')*beta[t+1][0]
        
        beta[t][1] =  ProbTransition[1][1]*ProbEmission[1][data[t+1]-1]*beta[t+1][1] \
                      + ProbTransition[1][0]*ProbEmission[0][data[t+1]-1]*beta[t+1][0]         

        # Normalization step
        beta[t][0],beta[t][1] = beta[t][0]/(beta[t][0]+beta[t][1]),beta[t][1]/(beta[t][0]+beta[t][1])
    
    #print(beta[124][0]/beta[124][1])
    #print(np.all(np.sum(beta,axis=1))==1)
    #print(beta[:5])
    return beta

In [30]:
def Baum_Welch(data):
    #Intialize Emission prob or b
    ProbEmission = np.array([[1/6,1/6,1/6,1/6,1/6,1/6],
                             [1/10,1/10,1/10,1/10,1/10,1/2]])
     
    #Initialize pi
    Pi = np.array([0.5,0.5])
    
    #Initialize transition probability
    ProbTransition = np.array([[0.95,0.05],
                               [0.05,0.95]])
    
    #Initialize tau
    tau = np.zeros(shape = (len(data),2,2))
    
    #Initialize gamma 
    gamma = np.zeros(shape = (len(data),2))
    
    #Initialize alpha and beta
    alpha_0 = np.zeros(shape=(len(data),2))
    beta_0 = np.zeros(shape=(len(data),2))
    
    for Iter in range(2000):
        if (Iter%100==0):
            print(Iter)
        #*********Compute the forward prob**************
        alpha  = forwardWB(data,ProbEmission,ProbTransition,alpha_0,Pi)
        
        #*********Compute the Backward prob*************
        beta   = backwardWB(data,ProbEmission,ProbTransition,beta_0,Pi)
        
        #print("\nalpha\n",alpha[:5])
        #print("\nbeta\n",beta[:5])
        #********** E step ******************************
        for t in range(len(data)-1):
            for i in range(2):
                gamma[t][i] = alpha[t][i]*beta[t][i]
                for j in range(2):
                    tau[t][i][j] = alpha[t][i]*ProbTransition[i][j]*beta[t+1][j]*ProbEmission[j][data[t+1]-1]

            #Normailzation step
            tau[t]   = tau[t]/np.sum(tau[t],axis=None)
            gamma[t] = gamma[t]/np.sum(gamma[t],axis=None)
            
        #********** M step *******************************
        
        Pi[0],Pi[1] = gamma[0][0],gamma[0][1]
        #print("\ngamma\n",gamma[:5])
        #print("\nsigma\n",tau[:5])
        
        #For Emission prob
        temp = np.zeros(shape=(2,6))
        temp2 = np.zeros(shape=(2,2))
        temp3 = np.zeros(shape=(2,1))
        for t in range(len(data)-1):
            num = data[t]
            temp[0][num-1] += gamma[t][0]
            temp[1][num-1] += gamma[t][1]
            
            temp2[0][0] += tau[t][0][0]
            temp2[0][1] += tau[t][0][1]
            temp2[1][0] += tau[t][1][0]
            temp2[1][1] += tau[t][1][1]
            
            temp3[0] += gamma[t][0]
            temp3[1] += gamma[t][1]
        
        ProbTransition = temp2/temp3
        
        ProbEmission = temp/np.sum(gamma,axis=0).reshape(2,1)
        #assert np.all(alpha_0 == alpha)
        #assert np.all(beta_0 == beta)
        #print(np.sum(ProbEmission,axis=1))
        #print(np.sum(ProbTransition,axis=1))
        alpha_0 = alpha.copy()
        beta_0 = beta.copy()
            
    return Pi,ProbEmission,ProbTransition

In [31]:
Pi,ProbEmission,ProbTransition = Baum_Welch(data_2)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [33]:
print("\nInitial Probablities are\n\n",Pi,"\n\nEmission Probabilities are\n\n",ProbEmission,"\n\nTransition Probabilities are \n\n",ProbTransition)


Initial Probablities are

 [1.e+000 3.e-323] 

Emission Probabilities are

 [[0.23011221 0.21110127 0.19111903 0.22936088 0.04854345 0.08976316]
 [0.1632448  0.19291674 0.18591227 0.16139451 0.19986771 0.09666398]] 

Transition Probabilities are 

 [[0.86360957 0.13639043]
 [0.14365604 0.85634396]]
